This requires sox and ffmpeg to be installed

In [2]:
CHUNK_SIZE      = 10 # Seconds
SAVE_SPECTO_AS  = 'png' # png or jpg. png is default
DELETE_WAV      = True # Deletes all wav files
NFFT            = 256 # Not sure yet: Length of the windowing segments
FS              = 256 # Not Sure yet: Sampling frequency

In [3]:
#import os
from pathlib import Path
#import glob
import subprocess
from pydub import AudioSegment
from pydub.utils import make_chunks
from scipy.io import wavfile
from matplotlib import pyplot as plt
from PIL import Image

In [4]:
def mp3ToWav(fpath, tmp_folder):
    file = Path(fpath)
    
    # Check file
    if not file.exists():
        return 1
    elif not file.suffix == '.mp3':
        return 2
    
    # Use sox to convert to 
    wav_file = (Path(tmp_folder)/file.stem).with_suffix('.wav')
    print("Exporting " + str(wav_file))
    subprocess.call(['sox', str(file), '-e', 'floating-point','-r', '16k', str(wav_file), 'remix', '1,2'])
    return wav_file

In [5]:
def wavToChunks(fpath, tmp_folder):
    file = Path(fpath)
    
    # Check file
    if not file.exists():
        return 1
    elif not file.suffix == '.wav':
        return 2
    
    myaudio = AudioSegment.from_file(file, 'wav')
    chunks  = make_chunks(myaudio, CHUNK_SIZE * 1000)
    
    tmp_folder = Path(tmp_folder)
    chunk_files = []
    for i,chunk in enumerate(chunks):
        chunk_name = tmp_folder/(file.stem + "_chunk{0}.wav".format(i))
        print("Exporting ", chunk_name)
        chunk.export(chunk_name, format="wav")
        chunk_files.append(chunk_name)

    return chunk_files

In [6]:
def mp3ToWavChunks(fpath, tmp_folder):
    file = Path(fpath)
    
    # Check file
    if not file.exists():
        return 1
    elif not file.suffix == '.mp3':
        return 2
    
    wav_file = mp3ToWav(fpath, tmp_folder)

    chunks = wavToChunks(wav_file, tmp_folder)
    wav_file.unlink()
    
    return chunks

In [7]:
def wavToSpectro(fpath, target_folder):
    file = Path(fpath)
    
    # Check file
    if not file.exists():
        return 1
    elif not file.suffix == '.wav':
        return 2
    
    rate, data = wavfile.read(file)
    
    plt.specgram(data, NFFT, FS)
    # Remove everything around the spectogram
    plt.axis('off')
    
    frame1 = plt.gca()
    for xlabel_i in frame1.axes.get_xticklabels():
        xlabel_i.set_visible(False)
        xlabel_i.set_fontsize(0.0)
    for xlabel_i in frame1.axes.get_yticklabels():
        xlabel_i.set_fontsize(0.0)
        xlabel_i.set_visible(False)
    for tick in frame1.axes.get_xticklines():
        tick.set_visible(False)
    for tick in frame1.axes.get_yticklines():
        tick.set_visible(False)
    
    output_file = target_folder+ "/" + file.stem + '.png'
    print("Exporting " + output_file)
    plt.savefig(output_file,
                dpi=100, # Dots per inch
                frameon='false',
                aspect='normal',
                bbox_inches='tight',
                pad_inches=0)
    plt.close()
    
    if SAVE_SPECTO_AS == "jpg":
        png = Path(output_file)
        im=Image.open(png)
        rgb_im=im.convert('RGB')
        rgb_im.save(png.with_suffix('.jpg'))
        png.unlink()        

In [20]:
file = './song.mp3'
#input_folders = ['D:\Music\Django Reinhardt\The Best of Django Reinhardt']
#input_folders = [#"D:\Music\Julia Fischer, Russian National Orchestra, Yakov Kreizberg\Tchaikovsky Violin Concerto, Serenade melancolique, Valse op.34 & Souvenir d'un lieu cher op.42",
                 #"D:\Music\Rachmaninoff Edition, The Complete Works - 2008 (31-Disc Box Set)\CD01 - Piano Concertos Nos. 1 & 4; Rhapsody (Paganini)",
                 #"D:\Music\Itzhak Perlman\Vivaldi-The Four Seasons and Violin Concertos",
                 #"D:\Music\Vladimir Horowitz\Favorite Chopin"]
#output_folder = 'classic'
input_folders = ["D:\Music\John Lee Hooker\Don't Look Back",
                  "D:\Music\\Nina Simone\Feeling Good_ The Very Best of Nina Simone",
                  "D:\Music\Sonny Boy Williamson\The Best Of",
                  "D:\Music\Charles Mingus\Mingus Ah Um [Remastered]",
                  "D:\Music\Dinah Washington\Verve Jazz Masters 19 - Dinah Washington"]
output_folder = "jazz"
tmp_folder = 'tmp'

if not Path(output_folder).is_dir():
    print("Creating folder: " + output_folder)
    Path(output_folder).mkdir()
    
if not Path(tmp_folder).is_dir():
    print("Creating folder: " + tmp_folder)
    Path(tmp_folder).mkdir()

for d in input_folders:
    for file in Path(d).iterdir():
        if  file.suffix == '.mp3':
            chunks = mp3ToWavChunks(file, tmp_folder)
            for chunk in chunks:
                wavToSpectro(chunk, output_folder)
                Path(chunk).unlink()

Exporting tmp\01 Dimples.wav
Exporting  tmp\01 Dimples_chunk0.wav
Exporting  tmp\01 Dimples_chunk1.wav
Exporting  tmp\01 Dimples_chunk2.wav
Exporting  tmp\01 Dimples_chunk3.wav
Exporting  tmp\01 Dimples_chunk4.wav
Exporting  tmp\01 Dimples_chunk5.wav
Exporting  tmp\01 Dimples_chunk6.wav
Exporting  tmp\01 Dimples_chunk7.wav
Exporting  tmp\01 Dimples_chunk8.wav
Exporting  tmp\01 Dimples_chunk9.wav
Exporting  tmp\01 Dimples_chunk10.wav
Exporting  tmp\01 Dimples_chunk11.wav
Exporting  tmp\01 Dimples_chunk12.wav
Exporting  tmp\01 Dimples_chunk13.wav
Exporting  tmp\01 Dimples_chunk14.wav
Exporting  tmp\01 Dimples_chunk15.wav
Exporting  tmp\01 Dimples_chunk16.wav
Exporting  tmp\01 Dimples_chunk17.wav
Exporting  tmp\01 Dimples_chunk18.wav
Exporting  tmp\01 Dimples_chunk19.wav
Exporting  tmp\01 Dimples_chunk20.wav
Exporting  tmp\01 Dimples_chunk21.wav
Exporting  tmp\01 Dimples_chunk22.wav
Exporting  tmp\01 Dimples_chunk23.wav
Exporting  tmp\01 Dimples_chunk24.wav
Exporting classic/01 Dimples_ch

Exporting classic/03 Ain't No Big Thing_chunk30.png
Exporting classic/03 Ain't No Big Thing_chunk31.png
Exporting classic/03 Ain't No Big Thing_chunk32.png
Exporting tmp\04 Don't Look Back.wav
Exporting  tmp\04 Don't Look Back_chunk0.wav
Exporting  tmp\04 Don't Look Back_chunk1.wav
Exporting  tmp\04 Don't Look Back_chunk2.wav
Exporting  tmp\04 Don't Look Back_chunk3.wav
Exporting  tmp\04 Don't Look Back_chunk4.wav
Exporting  tmp\04 Don't Look Back_chunk5.wav
Exporting  tmp\04 Don't Look Back_chunk6.wav
Exporting  tmp\04 Don't Look Back_chunk7.wav
Exporting  tmp\04 Don't Look Back_chunk8.wav
Exporting  tmp\04 Don't Look Back_chunk9.wav
Exporting  tmp\04 Don't Look Back_chunk10.wav
Exporting  tmp\04 Don't Look Back_chunk11.wav
Exporting  tmp\04 Don't Look Back_chunk12.wav
Exporting  tmp\04 Don't Look Back_chunk13.wav
Exporting  tmp\04 Don't Look Back_chunk14.wav
Exporting  tmp\04 Don't Look Back_chunk15.wav
Exporting  tmp\04 Don't Look Back_chunk16.wav
Exporting  tmp\04 Don't Look Back_c

Exporting  tmp\06 Spellbound_chunk17.wav
Exporting  tmp\06 Spellbound_chunk18.wav
Exporting  tmp\06 Spellbound_chunk19.wav
Exporting  tmp\06 Spellbound_chunk20.wav
Exporting  tmp\06 Spellbound_chunk21.wav
Exporting  tmp\06 Spellbound_chunk22.wav
Exporting  tmp\06 Spellbound_chunk23.wav
Exporting classic/06 Spellbound_chunk0.png
Exporting classic/06 Spellbound_chunk1.png
Exporting classic/06 Spellbound_chunk2.png
Exporting classic/06 Spellbound_chunk3.png
Exporting classic/06 Spellbound_chunk4.png
Exporting classic/06 Spellbound_chunk5.png
Exporting classic/06 Spellbound_chunk6.png
Exporting classic/06 Spellbound_chunk7.png
Exporting classic/06 Spellbound_chunk8.png
Exporting classic/06 Spellbound_chunk9.png
Exporting classic/06 Spellbound_chunk10.png
Exporting classic/06 Spellbound_chunk11.png
Exporting classic/06 Spellbound_chunk12.png
Exporting classic/06 Spellbound_chunk13.png
Exporting classic/06 Spellbound_chunk14.png
Exporting classic/06 Spellbound_chunk15.png
Exporting classic/0

Exporting classic/09 Frisco Blues_chunk9.png
Exporting classic/09 Frisco Blues_chunk10.png
Exporting classic/09 Frisco Blues_chunk11.png
Exporting classic/09 Frisco Blues_chunk12.png
Exporting classic/09 Frisco Blues_chunk13.png
Exporting classic/09 Frisco Blues_chunk14.png
Exporting classic/09 Frisco Blues_chunk15.png
Exporting classic/09 Frisco Blues_chunk16.png
Exporting classic/09 Frisco Blues_chunk17.png
Exporting classic/09 Frisco Blues_chunk18.png
Exporting classic/09 Frisco Blues_chunk19.png
Exporting classic/09 Frisco Blues_chunk20.png
Exporting classic/09 Frisco Blues_chunk21.png
Exporting classic/09 Frisco Blues_chunk22.png
Exporting tmp\10 Red House.wav
Exporting  tmp\10 Red House_chunk0.wav
Exporting  tmp\10 Red House_chunk1.wav
Exporting  tmp\10 Red House_chunk2.wav
Exporting  tmp\10 Red House_chunk3.wav
Exporting  tmp\10 Red House_chunk4.wav
Exporting  tmp\10 Red House_chunk5.wav
Exporting  tmp\10 Red House_chunk6.wav
Exporting  tmp\10 Red House_chunk7.wav
Exporting  tmp

Exporting  tmp\02 My Baby Just Cares for Me_chunk17.wav
Exporting  tmp\02 My Baby Just Cares for Me_chunk18.wav
Exporting classic/02 My Baby Just Cares for Me_chunk0.png
Exporting classic/02 My Baby Just Cares for Me_chunk1.png
Exporting classic/02 My Baby Just Cares for Me_chunk2.png
Exporting classic/02 My Baby Just Cares for Me_chunk3.png
Exporting classic/02 My Baby Just Cares for Me_chunk4.png
Exporting classic/02 My Baby Just Cares for Me_chunk5.png
Exporting classic/02 My Baby Just Cares for Me_chunk6.png
Exporting classic/02 My Baby Just Cares for Me_chunk7.png
Exporting classic/02 My Baby Just Cares for Me_chunk8.png
Exporting classic/02 My Baby Just Cares for Me_chunk9.png
Exporting classic/02 My Baby Just Cares for Me_chunk10.png
Exporting classic/02 My Baby Just Cares for Me_chunk11.png
Exporting classic/02 My Baby Just Cares for Me_chunk12.png
Exporting classic/02 My Baby Just Cares for Me_chunk13.png
Exporting classic/02 My Baby Just Cares for Me_chunk14.png
Exporting cla

...

Exporting  tmp\01 A Slick Chick On The Mellow Side_chunk14.wav
Exporting  tmp\01 A Slick Chick On The Mellow Side_chunk15.wav
Exporting  tmp\01 A Slick Chick On The Mellow Side_chunk16.wav
Exporting classic/01 A Slick Chick On The Mellow Side_chunk0.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk1.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk2.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk3.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk4.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk5.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk6.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk7.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk8.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk9.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk10.png
Exporting classic/01 A Slick Chick On The Mellow Side_chunk11.png
Exporting classic/01 A Slick 

Exporting classic/04 This Can't Be Love_chunk9.png
Exporting classic/04 This Can't Be Love_chunk10.png
Exporting classic/04 This Can't Be Love_chunk11.png
Exporting classic/04 This Can't Be Love_chunk12.png
Exporting classic/04 This Can't Be Love_chunk13.png
Exporting classic/04 This Can't Be Love_chunk14.png
Exporting classic/04 This Can't Be Love_chunk15.png
Exporting classic/04 This Can't Be Love_chunk16.png
Exporting classic/04 This Can't Be Love_chunk17.png
Exporting classic/04 This Can't Be Love_chunk18.png
Exporting classic/04 This Can't Be Love_chunk19.png
Exporting classic/04 This Can't Be Love_chunk20.png
Exporting classic/04 This Can't Be Love_chunk21.png
Exporting classic/04 This Can't Be Love_chunk22.png
Exporting classic/04 This Can't Be Love_chunk23.png
Exporting classic/04 This Can't Be Love_chunk24.png
Exporting classic/04 This Can't Be Love_chunk25.png
Exporting classic/04 This Can't Be Love_chunk26.png
Exporting classic/04 This Can't Be Love_chunk27.png
Exporting cla

Exporting  tmp\08 A Foggy Day_chunk36.wav
Exporting  tmp\08 A Foggy Day_chunk37.wav
Exporting  tmp\08 A Foggy Day_chunk38.wav
Exporting  tmp\08 A Foggy Day_chunk39.wav
Exporting  tmp\08 A Foggy Day_chunk40.wav
Exporting  tmp\08 A Foggy Day_chunk41.wav
Exporting  tmp\08 A Foggy Day_chunk42.wav
Exporting  tmp\08 A Foggy Day_chunk43.wav
Exporting  tmp\08 A Foggy Day_chunk44.wav
Exporting  tmp\08 A Foggy Day_chunk45.wav
Exporting  tmp\08 A Foggy Day_chunk46.wav
Exporting  tmp\08 A Foggy Day_chunk47.wav
Exporting  tmp\08 A Foggy Day_chunk48.wav
Exporting classic/08 A Foggy Day_chunk0.png
Exporting classic/08 A Foggy Day_chunk1.png
Exporting classic/08 A Foggy Day_chunk2.png
Exporting classic/08 A Foggy Day_chunk3.png
Exporting classic/08 A Foggy Day_chunk4.png
Exporting classic/08 A Foggy Day_chunk5.png
Exporting classic/08 A Foggy Day_chunk6.png
Exporting classic/08 A Foggy Day_chunk7.png
Exporting classic/08 A Foggy Day_chunk8.png
Exporting classic/08 A Foggy Day_chunk9.png
Exporting clas

Exporting classic/10 You Don't Know What Love Is_chunk16.png
Exporting classic/10 You Don't Know What Love Is_chunk17.png
Exporting classic/10 You Don't Know What Love Is_chunk18.png
Exporting classic/10 You Don't Know What Love Is_chunk19.png
Exporting classic/10 You Don't Know What Love Is_chunk20.png
Exporting classic/10 You Don't Know What Love Is_chunk21.png
Exporting classic/10 You Don't Know What Love Is_chunk22.png
Exporting classic/10 You Don't Know What Love Is_chunk23.png
Exporting classic/10 You Don't Know What Love Is_chunk24.png
Exporting tmp\11 Pennies From Heaven.wav
Exporting  tmp\11 Pennies From Heaven_chunk0.wav
Exporting  tmp\11 Pennies From Heaven_chunk1.wav
Exporting  tmp\11 Pennies From Heaven_chunk2.wav
Exporting  tmp\11 Pennies From Heaven_chunk3.wav
Exporting  tmp\11 Pennies From Heaven_chunk4.wav
Exporting  tmp\11 Pennies From Heaven_chunk5.wav
Exporting  tmp\11 Pennies From Heaven_chunk6.wav
Exporting  tmp\11 Pennies From Heaven_chunk7.wav
Exporting  tmp\11 

Exporting  tmp\15 Baby, You've Got What It Takes_chunk13.wav
Exporting  tmp\15 Baby, You've Got What It Takes_chunk14.wav
Exporting  tmp\15 Baby, You've Got What It Takes_chunk15.wav
Exporting  tmp\15 Baby, You've Got What It Takes_chunk16.wav
Exporting classic/15 Baby, You've Got What It Takes_chunk0.png
Exporting classic/15 Baby, You've Got What It Takes_chunk1.png
Exporting classic/15 Baby, You've Got What It Takes_chunk2.png
Exporting classic/15 Baby, You've Got What It Takes_chunk3.png
Exporting classic/15 Baby, You've Got What It Takes_chunk4.png
Exporting classic/15 Baby, You've Got What It Takes_chunk5.png
Exporting classic/15 Baby, You've Got What It Takes_chunk6.png
Exporting classic/15 Baby, You've Got What It Takes_chunk7.png
Exporting classic/15 Baby, You've Got What It Takes_chunk8.png
Exporting classic/15 Baby, You've Got What It Takes_chunk9.png
Exporting classic/15 Baby, You've Got What It Takes_chunk10.png
Exporting classic/15 Baby, You've Got What It Takes_chunk11.pn